[MAL API Documentation](https://myanimelist.net/apiconfig/references/api/v2#section/Versioning)

In [1]:
import json
import numpy as np
import os
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from helper.misc import canUpdateData, loadMALJSONIntoDF, createHOFDF, eligibleItems_Theme, summaryOfChallenge
from helper.download_data import myAnimeData, hofAnimeThemeData
from dateutil.relativedelta import relativedelta

In [2]:
dotenv_file = find_dotenv()
load_dotenv(dotenv_file)

print('Checking HOF Anime Data Import date...')
if canUpdateData(dotenv_file, 'nextHOFImportDate', '%Y-%m-%d', 12):
    themes_tags = ['aliens', 'androids', 'angels', 'angst', 'anthropomorphism', 'comingofage', 'conspiracy', 'deities', 'delinquents', 'detectives', 'disasters', 'dragons', 'elves', 'fairies', 'friendship', 'ghosts', 'gore', 'gunfights', 'historicalperiod', 'lovetriangles', 'maids', 'mythology', 'ninjas', 'parentalabandonment', 'powersuits', 'robots', 'slapstick', 'stereotypes', 'swordplay', 'tragedy', 'awardnotv', 'awardtv', 'timetravel', 'battleroyale', 'isekai']
    filePath = './data/HOF Data/'

    hofAnimeThemeData(themes_tags, filePath)

print()

print('Checking MAL Anime Data Import date...')
if canUpdateData(dotenv_file, 'nextMALImportDate_Anime', '%Y-%m-%d', 2):
    clientId = os.getenv('X-MAL-CLIENT-ID')
    animeList = os.getenv('myAnimeList')
    fields = ['rating, media_type, genres, studios, list_status']
    filePath = './data/MAL Anime Data/'
    myAnimeData(clientId, animeList, fields, filePath)

Checking HOF Anime Data Import date...
Cannot update data. Next scheduled update is 2022-08-23

Checking MAL Anime Data Import date...
Cannot update data. Next scheduled update is 2022-06-13


In [3]:
with open("./data/MAL Anime Data/watching.json", "r") as file:
    watchingJson = json.load(file)

with open("./data/MAL Anime Data/completed.json", "r") as file:
    completedJson = json.load(file)

with open("./data/HOF Data/hofTheme.json", "r") as file:
    hofThemeJson = json.load(file)

In [4]:
nodeColumns = ['id', 'title', 'genres', 'rating', 'media_type', 'studios']
listStatusColumns = ['status', 'start_date', 'finish_date']
other = ['color', 'label']

myMALDF_Anime = pd.DataFrame(columns=nodeColumns+listStatusColumns)

myMALDF_Anime = loadMALJSONIntoDF(watchingJson, nodeColumns, listStatusColumns, myMALDF_Anime)
myMALDF_Anime = loadMALJSONIntoDF(completedJson, nodeColumns, listStatusColumns, myMALDF_Anime)
myMALDF_Anime['color'] = 'N/A'
myMALDF_Anime['label'] = 'N/A'
myMALDF_Anime = myMALDF_Anime.astype({'id' : 'int64', 'title' : 'string', 'rating' : 'string', 'media_type' : 'string', 'status' : 'string', 'start_date' : 'string', 'finish_date' : 'string', 'color' : 'string', 'label' : 'string'})
myMALDF_Anime

,id,title,genres,rating,media_type,studios,status,start_date,finish_date,color,label
0,40904,Bokutachi no Remake,"[Drama, Romance, Slice of Life, Time Travel]",pg_13,tv,[feel.],watching,2022-03-23,N/A,N/A,N/A
1,40623,Dokyuu Hentai HxEros,"[Action, Comedy, Ecchi, School, Shounen, Super...",r+,tv,[Project No.9],watching,N/A,N/A,N/A,N/A
2,49854,Getsuyoubi no Tawawa 2,"[Ecchi, Slice of Life]",pg_13,ona,[Yokohama Animation Lab],watching,N/A,N/A,N/A,N/A
3,39463,Gleipnir,"[Action, Ecchi, Mystery, Seinen, Supernatural]",r,tv,[Pine Jam],watching,2022-01-08,N/A,N/A,N/A
4,40746,Overflow,"[Hentai, Romance]",rx,ona,[Studio Hokiboshi],watching,2022-01-23,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...
442,1840,Zero no Tsukaima: Futatsuki no Kishi,"[Action, Adventure, Comedy, Ecchi, Fantasy, Ha...",pg_13,tv,[J.C.Staff],completed,2014-11-22,2014-11-22,N/A,N/A
443,3712,Zero no Tsukaima: Princesses no Rondo,"[Action, Adventure, Comedy, Ecchi, Fantasy, Ha...",pg_13,tv,[J.C.Staff],completed,2014-11-23,2014-11-23,N/A,N/A
444,5764,Zero no Tsukaima: Princesses no Rondo - Yuuwak...,"[Comedy, Ecchi, Fantasy, Harem, Isekai]",pg_13,special,[J.C.Staff],completed,2016-10-08,2016-10-08,N/A,N/A
445,712,Zoids Genesis,"[Action, Adventure, Comedy, Mecha, Military]",pg,tv,[Shogakukan Music & Digital Entertainment],completed,2011-07-19,2011-07-31,N/A,N/A


In [5]:
myHOFAnimeDF = createHOFDF(hofThemeJson, myMALDF_Anime)
myHOFAnimeDF

,mal_id,themes,title,status,finish_date,color,label
0,5337,[aliens],Bakugan Battle Brawlers: New Vestroia,completed,2010-06-06,N/A,N/A
1,530,"[aliens, lovetriangles, ninjas, slapstick, awa...",Bishoujo Senshi Sailor Moon,completed,2006-07-31,N/A,N/A
2,740,"[aliens, androids, deities, lovetriangles, sla...",Bishoujo Senshi Sailor Moon R,completed,2007-02-17,N/A,N/A
3,35849,"[aliens, comingofage, conspiracy, disasters, g...",Darling in the FranXX,completed,2018-12-22,N/A,N/A
4,15583,"[aliens, gunfights, powersuits, swordplay]",Date A Live,completed,2013-12-28,N/A,N/A
...,...,...,...,...,...,...,...
378,36456,[awardtv],Boku no Hero Academia 3rd Season,completed,2018-09-30,N/A,N/A
379,19111,[awardtv],Love Live! School Idol Project 2nd Season,completed,2016-03-31,N/A,N/A
380,40206,[timetravel],Fate/Grand Order: Zettai Majuu Sensen Babyloni...,completed,2021-05-21,N/A,N/A
381,6951,[timetravel],Yu☆Gi☆Oh! Movie: Chou Yuugou! Toki wo Koeta Ki...,completed,N/A,N/A,N/A


# Studio Challenge

This challenge is similar to the pre-existing studio challenge, but rather than forcing you to watch only one studio, it aims at exposing you to a lot of studios (40, to be exact) giving members the opportunity to find a studio they have never knew of before.

　This challenge is similar to the pre-existing studio challenge, but rather than forcing you to watch only one studio, it aims at exposing you to a lot of studios (40, to be exact) giving members the opportunity to find a studio they have never knew of before.

HOW-TO:
- Find the Studios List [here](https://myanimelist.net/anime/producer).
- You can use any studio, but your challenge must contain 40 different studios.
- Studios that are clearly "part" of another studio count as only 1 studio.
- Example: Bandai includes > Bandai, Bandai Channel, Bandai Entertainment, etc.


Difficulty: complete a list of 40 different studios.
- If an anime has two or more studios, it can only count towards one for the challenge.
- This challenge can be completed thrice (x3). (added on Nov 05, 2019)

Previously completed series aren't allowed. Neither is re-watching.
- Series previously started, but not finished, may be used.
- Airing can be used, but you must complete the series, so you can only turn-in the challenge after the series has finished airing.

Only TV types, of any length are allowed.

Your challenge "Starting Date" is the day you post.
- Remember that it's required to add the start and end dates to any anime you use for our challenges.

You cannot use the same entry more than once across all difficulties.

To make this easier on the inspectors, you MUST list the studios in alphabetical order. Otherwise it's an instant invalid.

[Challenge Prompt](https://myanimelist.net/forum/?topicid=1504823)

[My Challenge Form](https://myanimelist.net/forum/?topicid=1504823&show=400#msg60311165)

In [6]:
studioChallengeDate = '2020-07-25'
notEligibleMediaTypes = ['ona', 'movie', 'special', 'music', 'ova']

studiosDF = myMALDF_Anime.loc[:, ['id', 'title', 'status', 'media_type', 'studios', 'finish_date', 'color', 'label']]

indexValues = set()

indexValues.update(studiosDF.index[studiosDF['finish_date'] == 'N/A'].intersection(studiosDF.index[studiosDF['status'] == 'completed']))
indexValues.update(studiosDF.index[studiosDF['media_type'].isin(notEligibleMediaTypes)])
indexValues.update(studiosDF.index[studiosDF['finish_date'] < studioChallengeDate].intersection(studiosDF.index[studiosDF['status'] == 'completed']))
indexValues.update(studiosDF.index[~studiosDF['studios'].astype(bool)])

studiosDF.drop(indexValues, axis='index', inplace=True)

studiosDF.sort_values('finish_date', inplace=True, ignore_index=True)

studiosUsed = []
indexValues = []

for rowIndex in range(studiosDF.shape[0]):
    res = list(set(studiosDF.iloc[rowIndex]['studios']).difference(studiosUsed))

    if res:
        studiosUsed.append(res[0])
        studiosDF.at[rowIndex, 'studios'] = res[0]
        studiosDF.at[rowIndex, 'color'] = 'green' if studiosDF.at[rowIndex, 'status'] == 'completed' else 'orange'
        studiosDF.at[rowIndex, 'label'] = 'o' if studiosDF.at[rowIndex, 'status'] == 'completed' else 'x'
    else:
        indexValues.append(rowIndex)

studiosDF.drop(index=indexValues, columns=['media_type', 'finish_date'], inplace=True)
studiosDF.sort_values('studios', inplace=True, ignore_index=True)

numberOfCompleted = len(studiosDF.index[studiosDF['status'] == 'completed'])

studiosDF[0:40]

,id,title,status,studios,color,label
0,40540,Sword Art Online: Alicization - War of Underwo...,completed,A-1 Pictures,green,o
1,41402,Mairimashita! Iruma-kun 2nd Season,completed,Bandai Namco Pictures,green,o
2,39783,5-toubun no Hanayome ∬,completed,Bibury Animation Studios,green,o
3,39017,Kyokou Suiri,completed,Brain's Base,green,o
4,40930,Azur Lane: Bisoku Zenshin!,completed,CANDY BOX,green,o
5,36023,Persona 5 the Animation,completed,CloverWorks,green,o
6,39292,Princess Connect! Re:Dive,completed,CygamesPictures,green,o
7,12293,Campione!: Matsurowanu Kamigami to Kamigoroshi...,completed,Diomedéa,green,o
8,36470,Tada-kun wa Koi wo Shinai,completed,Doga Kobo,green,o
9,41226,Uzaki-chan wa Asobitai!,completed,ENGI,green,o


# Theme Tags Challenge

Considering that there are so many interesting tags and themes out there used in anime, we have decided to make this challenge to group them all. This challenge will consist of several approved tags used to describe themes that appear in different series, and instead of releasing individual challenges for them (see Iyashikei, GAR, CGI, etc.) you can complete their version of this challenge here.

To suggest a tag, make sure there are enough series listed under them and suggest them on discord or in the Suggestion & Discussion thread. (Other requirements may be added later on)

Difficulty:
- All versions will need 40 series in total.
- No x2+ should be attempted for this challenge. Points won't be awarded.

Previously seen is not allowed.
- Series previously started, but not finished, may be used.
- Airing can be used, but you must complete the series, so you can only turn-in the challenge after the series has finished airing.
- Note: If you've seen all available for a certain tag, please be sure to contact one of the staff to see what your options are.

Any type is allowed, but must be equal to or longer than 15 minutes (total).

Start date is the day you post.
- NOTE that since new tags are added after the challenge release date, you can't use an old post for them. Once a new tag is added, you have to make a post for it.
- Newer tags can't be started before they were added.

Cannot use the same series and/or entry twice in one version, but you can use the same series for different versions.
- For example: "Cross Ange: Tenshi to Ryuu no Rondo" appears tagged as "Dragons" and "Rebellions" in Anime-Planet.
- If both tags were accepted as versions of this challenge, you could use them for both.

Limit yourself to use the provided sites for each tag. If you were to find another suitable listing, provide it to a staff so we can verify it.

This challenge is available in our [Hall of Fame](https://anime.jhiday.net/hof/challengeList#category-16) as a checklist that will help you complete it.
- You will need to have turned-in at least once (any challenge) to appear listed. Check if you are in the database here.

[Challenge Prompt](https://myanimelist.net/forum/?topicid=1802693)

[My Challenge Form – Set 1](https://myanimelist.net/forum/?topicid=1802693&show=150#msg60283622)

[My Challenge Form – Set 2](https://myanimelist.net/forum/?topicid=1802693&show=250#msg62428999)

### Aliens

In [7]:
aliensDF = eligibleItems_Theme(myHOFAnimeDF, 'aliens', '2020-07-23')
aliensDF

,mal_id,title,status,finish_date,color,label
0,8726,Soredemo Machi wa Mawatteiru,completed,2020-08-22,green,o
1,36633,Date A Live III,completed,2021-01-02,green,o
2,49357,Star Wars: Visions,completed,2021-09-23,green,o
3,40623,Dokyuu Hentai HxEros,watching,N/A,orange,x
4,39463,Gleipnir,watching,N/A,orange,x


### Angst

In [8]:
angstDF = eligibleItems_Theme(myHOFAnimeDF, 'angst', '2020-07-23')
angstDF

,mal_id,title,status,finish_date,color,label
0,39017,Kyokou Suiri,completed,2020-08-04,green,o
1,164,Mononoke Hime,completed,2020-09-17,green,o
2,40530,Jaku-Chara Tomozaki-kun,completed,2021-03-26,green,o
3,40748,Jujutsu Kaisen (TV),completed,2021-04-19,green,o
4,42897,Horimiya,completed,2021-04-24,green,o
5,40938,Hige wo Soru. Soshite Joshikousei wo Hirou.,completed,2021-06-28,green,o
6,41457,86,completed,2021-06-30,green,o
7,45604,Jaku-Chara Tomozaki-kun Specials,completed,2021-07-04,green,o
8,41402,Mairimashita! Iruma-kun 2nd Season,completed,2021-09-11,green,o
9,49605,Ganbare Douki-chan,completed,2022-01-07,green,o


### Anthropomorphism

In [9]:
anthropomorphismDF = eligibleItems_Theme(myHOFAnimeDF, 'anthropomorphism', '2020-07-23')
anthropomorphismDF

,mal_id,title,status,finish_date,color,label
0,164,Mononoke Hime,completed,2020-09-17,green,o
1,39535,Mushoku Tensei: Isekai Ittara Honki Dasu,completed,2021-03-21,green,o
2,40930,Azur Lane: Bisoku Zenshin!,completed,2021-04-06,green,o
3,40748,Jujutsu Kaisen (TV),completed,2021-04-19,green,o
4,41457,86,completed,2021-06-30,green,o
5,41168,Nakitai Watashi wa Neko wo Kaburu,completed,2022-02-17,green,o


### Deities

In [10]:
deitiesDF = eligibleItems_Theme(myHOFAnimeDF, 'deities', '2020-07-23')
deitiesDF

,mal_id,title,status,finish_date,color,label
0,39017,Kyokou Suiri,completed,2020-08-04,green,o
1,164,Mononoke Hime,completed,2020-09-17,green,o
2,40540,Sword Art Online: Alicization - War of Underwo...,completed,2020-09-20,green,o
3,12293,Campione!: Matsurowanu Kamigami to Kamigoroshi...,completed,2020-11-12,green,o
4,17074,Monogatari Series: Second Season,completed,2020-11-30,green,o
5,40496,Maou Gakuin no Futekigousha: Shijou Saikyou no...,completed,2021-01-05,green,o
6,39535,Mushoku Tensei: Isekai Ittara Honki Dasu,completed,2021-03-21,green,o
7,40064,Dungeon ni Deai wo Motomeru no wa Machigatteir...,completed,2021-05-25,green,o
8,32801,Dungeon ni Deai wo Motomeru no wa Machigatteir...,completed,2021-05-25,green,o


### Detectives

In [11]:
detectivesDF = eligibleItems_Theme(myHOFAnimeDF, 'detectives', '2020-07-23')
detectivesDF

,mal_id,title,status,finish_date,color,label
0,39017,Kyokou Suiri,completed,2020-08-04,green,o
1,39198,Kanata no Astra,completed,2020-08-13,green,o
2,8726,Soredemo Machi wa Mawatteiru,completed,2020-08-22,green,o
3,36023,Persona 5 the Animation,completed,2020-10-17,green,o
4,40496,Maou Gakuin no Futekigousha: Shijou Saikyou no...,completed,2021-01-05,green,o
5,39326,Kawaikereba Hentai demo Suki ni Natte Kuremasu...,completed,2021-04-12,green,o


### Dragons

In [12]:
dragonsDF = eligibleItems_Theme(myHOFAnimeDF, 'dragons', '2020-07-23')
dragonsDF

,mal_id,title,status,finish_date,color,label
0,40540,Sword Art Online: Alicization - War of Underwo...,completed,2020-09-20,green,o
1,41468,Burn the Witch,completed,2021-01-02,green,o
2,41402,Mairimashita! Iruma-kun 2nd Season,completed,2021-09-11,green,o


### Fairies

In [13]:
fairiesDF = eligibleItems_Theme(myHOFAnimeDF, 'fairies', '2020-07-23')
fairiesDF

,mal_id,title,status,finish_date,color,label
0,40540,Sword Art Online: Alicization - War of Underwo...,completed,2020-09-20,green,o


### Ghosts

In [14]:
ghostsDF = eligibleItems_Theme(myHOFAnimeDF, 'ghosts', '2020-07-23')
ghostsDF

,mal_id,title,status,finish_date,color,label
0,39017,Kyokou Suiri,completed,2020-08-04,green,o
1,39292,Princess Connect! Re:Dive,completed,2020-10-08,green,o
2,17074,Monogatari Series: Second Season,completed,2020-11-30,green,o
3,47257,Shinigami Bocchan to Kuro Maid,completed,2021-09-19,green,o


### Historical Period

In [15]:
historicalperiodDF = eligibleItems_Theme(myHOFAnimeDF, 'historicalperiod', '2020-07-23')
historicalperiodDF

,mal_id,title,status,finish_date,color,label
0,164,Mononoke Hime,completed,2020-09-17,green,o


### Love Triangles

In [16]:
lovetrianglesDF = eligibleItems_Theme(myHOFAnimeDF, 'lovetriangles', '2020-07-23')
lovetrianglesDF

,mal_id,title,status,finish_date,color,label
0,36470,Tada-kun wa Koi wo Shinai,completed,2020-07-25,green,o
1,40839,"Kanojo, Okarishimasu",completed,2020-09-26,green,o
2,40436,Peter Grill to Kenja no Jikan,completed,2021-01-11,green,o
3,42897,Horimiya,completed,2021-04-24,green,o
4,40938,Hige wo Soru. Soshite Joshikousei wo Hirou.,completed,2021-06-28,green,o
5,43969,Kanojo mo Kanojo,completed,2021-09-19,green,o
6,49605,Ganbare Douki-chan,completed,2022-01-07,green,o
7,40623,Dokyuu Hentai HxEros,watching,N/A,orange,x
8,39463,Gleipnir,watching,N/A,orange,x


### Mythology

In [17]:
mythologyDF = eligibleItems_Theme(myHOFAnimeDF, 'mythology', '2020-07-23')
mythologyDF

,mal_id,title,status,finish_date,color,label
0,28701,Fate/stay night: Unlimited Blade Works 2nd Season,completed,2020-08-01,green,o
1,39017,Kyokou Suiri,completed,2020-08-04,green,o
2,164,Mononoke Hime,completed,2020-09-17,green,o
3,12293,Campione!: Matsurowanu Kamigami to Kamigoroshi...,completed,2020-11-12,green,o
4,41468,Burn the Witch,completed,2021-01-02,green,o
5,34321,Fate/Grand Order: First Order,completed,2021-04-30,green,o
6,40064,Dungeon ni Deai wo Motomeru no wa Machigatteir...,completed,2021-05-25,green,o
7,25537,Fate/stay night Movie: Heaven's Feel - I. Pres...,completed,2021-08-07,green,o
8,33049,Fate/stay night Movie: Heaven's Feel - II. Los...,completed,2021-08-08,green,o


### Ninjas

In [18]:
ninjasDF = eligibleItems_Theme(myHOFAnimeDF, 'ninjas', '2020-07-23')
ninjasDF

,mal_id,title,status,finish_date,color,label
0,33926,Quanzhi Gaoshou,watching,N/A,orange,x


### Robots

In [19]:
robotsDF = eligibleItems_Theme(myHOFAnimeDF, 'robots', '2020-07-23')
robotsDF

,mal_id,title,status,finish_date,color,label
0,40436,Peter Grill to Kenja no Jikan,completed,2021-01-11,green,o
1,41457,86,completed,2021-06-30,green,o
2,49357,Star Wars: Visions,completed,2021-09-23,green,o


### Slapstick Humor

In [20]:
slapstickDF = eligibleItems_Theme(myHOFAnimeDF, 'slapstick', '2020-07-23')
slapstickDF

,mal_id,title,status,finish_date,color,label
0,39017,Kyokou Suiri,completed,2020-08-04,green,o
1,12293,Campione!: Matsurowanu Kamigami to Kamigoroshi...,completed,2020-11-12,green,o
2,17074,Monogatari Series: Second Season,completed,2020-11-30,green,o
3,40436,Peter Grill to Kenja no Jikan,completed,2021-01-11,green,o
4,39535,Mushoku Tensei: Isekai Ittara Honki Dasu,completed,2021-03-21,green,o
5,39783,5-toubun no Hanayome ∬,completed,2021-03-26,green,o
6,40930,Azur Lane: Bisoku Zenshin!,completed,2021-04-06,green,o
7,42897,Horimiya,completed,2021-04-24,green,o
8,43007,Osananajimi ga Zettai ni Makenai Love Comedy,completed,2021-08-31,green,o
9,41402,Mairimashita! Iruma-kun 2nd Season,completed,2021-09-11,green,o


### Stereotypes

In [21]:
stereotypesDF = eligibleItems_Theme(myHOFAnimeDF, 'stereotypes', '2020-07-23')
stereotypesDF

,mal_id,title,status,finish_date,color,label
0,39026,Dumbbell Nan Kilo Moteru?,completed,2020-07-27,green,o
1,39017,Kyokou Suiri,completed,2020-08-04,green,o
2,36098,Kimi no Suizou wo Tabetai,completed,2020-11-05,green,o
3,12293,Campione!: Matsurowanu Kamigami to Kamigoroshi...,completed,2020-11-12,green,o
4,39783,5-toubun no Hanayome ∬,completed,2021-03-26,green,o
5,40930,Azur Lane: Bisoku Zenshin!,completed,2021-04-06,green,o
6,39326,Kawaikereba Hentai demo Suki ni Natte Kuremasu...,completed,2021-04-12,green,o
7,41402,Mairimashita! Iruma-kun 2nd Season,completed,2021-09-11,green,o
8,47257,Shinigami Bocchan to Kuro Maid,completed,2021-09-19,green,o
9,49605,Ganbare Douki-chan,completed,2022-01-07,green,o


### Swordplay

In [22]:
swordplayDF = eligibleItems_Theme(myHOFAnimeDF, 'swordplay', '2020-07-23')
swordplayDF

,mal_id,title,status,finish_date,color,label
0,28701,Fate/stay night: Unlimited Blade Works 2nd Season,completed,2020-08-01,green,o
1,164,Mononoke Hime,completed,2020-09-17,green,o
2,40540,Sword Art Online: Alicization - War of Underwo...,completed,2020-09-20,green,o
3,39292,Princess Connect! Re:Dive,completed,2020-10-08,green,o
4,12293,Campione!: Matsurowanu Kamigami to Kamigoroshi...,completed,2020-11-12,green,o
5,17074,Monogatari Series: Second Season,completed,2020-11-30,green,o
6,40496,Maou Gakuin no Futekigousha: Shijou Saikyou no...,completed,2021-01-05,green,o
7,40436,Peter Grill to Kenja no Jikan,completed,2021-01-11,green,o
8,39535,Mushoku Tensei: Isekai Ittara Honki Dasu,completed,2021-03-21,green,o
9,40748,Jujutsu Kaisen (TV),completed,2021-04-19,green,o


### Tragedy

In [23]:
tragedyDF = eligibleItems_Theme(myHOFAnimeDF, 'tragedy', '2020-07-23')
tragedyDF

,mal_id,title,status,finish_date,color,label
0,39017,Kyokou Suiri,completed,2020-08-04,green,o
1,164,Mononoke Hime,completed,2020-09-17,green,o
2,36098,Kimi no Suizou wo Tabetai,completed,2020-11-05,green,o
3,40496,Maou Gakuin no Futekigousha: Shijou Saikyou no...,completed,2021-01-05,green,o
4,40748,Jujutsu Kaisen (TV),completed,2021-04-19,green,o
5,34321,Fate/Grand Order: First Order,completed,2021-04-30,green,o
6,41457,86,completed,2021-06-30,green,o
7,47257,Shinigami Bocchan to Kuro Maid,completed,2021-09-19,green,o
8,40623,Dokyuu Hentai HxEros,watching,N/A,orange,x
9,39463,Gleipnir,watching,N/A,orange,x


### Androids

In [24]:
androidsDF = eligibleItems_Theme(myHOFAnimeDF, 'androids', '2021-03-21')
androidsDF

,mal_id,title,status,finish_date,color,label
0,49357,Star Wars: Visions,completed,2021-09-23,green,o


### Angels

In [25]:
angelsDF = eligibleItems_Theme(myHOFAnimeDF, 'angels', '2021-03-21')
angelsDF

,mal_id,title,status,finish_date,color,label
0,39326,Kawaikereba Hentai demo Suki ni Natte Kuremasu...,completed,2021-04-12,green,o


### Coming of Age

In [26]:
comingofageDF = eligibleItems_Theme(myHOFAnimeDF, 'comingofage', '2021-03-21')
comingofageDF

,mal_id,title,status,finish_date,color,label
0,39535,Mushoku Tensei: Isekai Ittara Honki Dasu,completed,2021-03-21,green,o
1,42897,Horimiya,completed,2021-04-24,green,o
2,41402,Mairimashita! Iruma-kun 2nd Season,completed,2021-09-11,green,o
3,47257,Shinigami Bocchan to Kuro Maid,completed,2021-09-19,green,o
4,40623,Dokyuu Hentai HxEros,watching,N/A,orange,x
5,39463,Gleipnir,watching,N/A,orange,x
6,40904,Bokutachi no Remake,watching,N/A,orange,x
7,49854,Getsuyoubi no Tawawa 2,watching,N/A,orange,x


### Conspiracy

In [27]:
conspiracyDF = eligibleItems_Theme(myHOFAnimeDF, 'conspiracy', '2021-03-21')
conspiracyDF

,mal_id,title,status,finish_date,color,label
0,33926,Quanzhi Gaoshou,watching,N/A,orange,x


### Delinquents

In [28]:
delinquentsDF = eligibleItems_Theme(myHOFAnimeDF, 'friendship', '2021-03-21')
delinquentsDF

,mal_id,title,status,finish_date,color,label
0,40530,Jaku-Chara Tomozaki-kun,completed,2021-03-26,green,o
1,40930,Azur Lane: Bisoku Zenshin!,completed,2021-04-06,green,o
2,40748,Jujutsu Kaisen (TV),completed,2021-04-19,green,o
3,42897,Horimiya,completed,2021-04-24,green,o
4,45604,Jaku-Chara Tomozaki-kun Specials,completed,2021-07-04,green,o
5,41402,Mairimashita! Iruma-kun 2nd Season,completed,2021-09-11,green,o
6,47257,Shinigami Bocchan to Kuro Maid,completed,2021-09-19,green,o
7,41168,Nakitai Watashi wa Neko wo Kaburu,completed,2022-02-17,green,o
8,39463,Gleipnir,watching,N/A,orange,x
9,40904,Bokutachi no Remake,watching,N/A,orange,x


### Disasters

In [29]:
disastersDF = eligibleItems_Theme(myHOFAnimeDF, 'disasters', '2021-03-21')
disastersDF

,mal_id,title,status,finish_date,color,label
0,39463,Gleipnir,watching,N/A,orange,x


### Elves

In [30]:
elvesDF = eligibleItems_Theme(myHOFAnimeDF, 'elves', '2021-03-21')
elvesDF

,mal_id,title,status,finish_date,color,label
0,39535,Mushoku Tensei: Isekai Ittara Honki Dasu,completed,2021-03-21,green,o
1,40064,Dungeon ni Deai wo Motomeru no wa Machigatteir...,completed,2021-05-25,green,o
2,32801,Dungeon ni Deai wo Motomeru no wa Machigatteir...,completed,2021-05-25,green,o


### Friendship

In [31]:
friendshipDF = eligibleItems_Theme(myHOFAnimeDF, 'friendship', '2021-03-21')
friendshipDF

,mal_id,title,status,finish_date,color,label
0,40530,Jaku-Chara Tomozaki-kun,completed,2021-03-26,green,o
1,40930,Azur Lane: Bisoku Zenshin!,completed,2021-04-06,green,o
2,40748,Jujutsu Kaisen (TV),completed,2021-04-19,green,o
3,42897,Horimiya,completed,2021-04-24,green,o
4,45604,Jaku-Chara Tomozaki-kun Specials,completed,2021-07-04,green,o
5,41402,Mairimashita! Iruma-kun 2nd Season,completed,2021-09-11,green,o
6,47257,Shinigami Bocchan to Kuro Maid,completed,2021-09-19,green,o
7,41168,Nakitai Watashi wa Neko wo Kaburu,completed,2022-02-17,green,o
8,39463,Gleipnir,watching,N/A,orange,x
9,40904,Bokutachi no Remake,watching,N/A,orange,x


### Gore

In [32]:
goreDF = eligibleItems_Theme(myHOFAnimeDF, 'gore', '2021-03-21')
goreDF

,mal_id,title,status,finish_date,color,label
0,39535,Mushoku Tensei: Isekai Ittara Honki Dasu,completed,2021-03-21,green,o
1,40748,Jujutsu Kaisen (TV),completed,2021-04-19,green,o
2,32801,Dungeon ni Deai wo Motomeru no wa Machigatteir...,completed,2021-05-25,green,o
3,39463,Gleipnir,watching,N/A,orange,x


### Gunfights

In [33]:
gunfightsDF = eligibleItems_Theme(myHOFAnimeDF, 'gunfights', '2021-03-21')
gunfightsDF

,mal_id,title,status,finish_date,color,label
0,33926,Quanzhi Gaoshou,watching,N/A,orange,x


### Maids

In [34]:
maidsDF = eligibleItems_Theme(myHOFAnimeDF, 'maids', '2021-03-21')
maidsDF

,mal_id,title,status,finish_date,color,label
0,39535,Mushoku Tensei: Isekai Ittara Honki Dasu,completed,2021-03-21,green,o
1,43609,Kaguya-sama wa Kokurasetai: Tensai-tachi no Re...,completed,2021-06-28,green,o
2,37021,Iya na Kao sare nagara Opantsu Misete Moraitai,completed,2021-07-04,green,o
3,38163,Iya na Kao sare nagara Opantsu Misete Moraitai 2,completed,2021-07-04,green,o
4,25537,Fate/stay night Movie: Heaven's Feel - I. Pres...,completed,2021-08-07,green,o
5,33049,Fate/stay night Movie: Heaven's Feel - II. Los...,completed,2021-08-08,green,o
6,47257,Shinigami Bocchan to Kuro Maid,completed,2021-09-19,green,o


### Parental Abadonment

In [35]:
parentalabandonmentDF = eligibleItems_Theme(myHOFAnimeDF, 'parentalabandonment', '2021-03-21')
parentalabandonmentDF

,mal_id,title,status,finish_date,color,label
0,39326,Kawaikereba Hentai demo Suki ni Natte Kuremasu...,completed,2021-04-12,green,o
1,42897,Horimiya,completed,2021-04-24,green,o
2,40938,Hige wo Soru. Soshite Joshikousei wo Hirou.,completed,2021-06-28,green,o
3,41402,Mairimashita! Iruma-kun 2nd Season,completed,2021-09-11,green,o
4,47257,Shinigami Bocchan to Kuro Maid,completed,2021-09-19,green,o
5,41168,Nakitai Watashi wa Neko wo Kaburu,completed,2022-02-17,green,o
6,40623,Dokyuu Hentai HxEros,watching,N/A,orange,x
7,39463,Gleipnir,watching,N/A,orange,x


### Power Suits

In [36]:
powersuitsDF = eligibleItems_Theme(myHOFAnimeDF, 'powersuits', '2021-03-21')
powersuitsDF

,mal_id,title,status,finish_date,color,label
0,39463,Gleipnir,watching,N/A,orange,x


# Summary

In [37]:
summaryDF = pd.DataFrame(columns=['challenge_name', 'progress', 'difficulty', 'items_added', 'items_removed', 'update', 'complete'])
challenges = ['studios', 'aliens', 'androids', 'angels', 'angst', 'anthropomorphism', 'comingofage', 'conspiracy', 'deities', 'delinquents', 'detectives', 'disasters', 'dragons', 'elves', 'fairies', 'friendship', 'ghosts', 'gore', 'gunfights', 'historicalperiod', 'lovetriangles', 'maids', 'mythology', 'ninjas', 'parentalabandonment', 'powersuits', 'robots', 'slapstick', 'stereotypes', 'swordplay', 'tragedy']

for challenge in challenges:
    summaryDF = pd.concat([summaryOfChallenge(challenge_name=challenge, difficulty='N/A', numberOfExpectedItems=40, animeDF=locals()[f'{challenge}DF']), summaryDF], ignore_index=True)

summaryDF.sort_values('challenge_name', inplace=True, ignore_index=True)

summaryDF

,challenge_name,progress,difficulty,items_added,items_removed,update,complete
0,aliens,3/40,N/A,N/A,N/A,N/A,False
1,androids,1/40,N/A,N/A,N/A,N/A,False
2,angels,1/40,N/A,N/A,N/A,N/A,False
3,angst,10/40,N/A,N/A,N/A,N/A,False
4,anthropomorphism,6/40,N/A,N/A,N/A,N/A,False
5,comingofage,4/40,N/A,N/A,N/A,N/A,False
6,conspiracy,0/40,N/A,N/A,N/A,N/A,False
7,deities,9/40,N/A,N/A,N/A,N/A,False
8,delinquents,8/40,N/A,N/A,N/A,N/A,False
9,detectives,6/40,N/A,N/A,N/A,N/A,False


# END